## Rodando os modelos para uma coleção de imagens Landsat 

#### Este exemplo apresenta como iniciar o modelo ssebop e criar uma classe "Collection" para uma coleção de imagens Landsat utilizando a função ".overpass()"

### Importando a biblioteca Google Earth Engine

In [1]:
try:
    import ee 
except:
    !pip install earthengine-api
    import ee

In [2]:
# Instalando bibliotecas secundárias do OpenET

# Installing openet-core package
# https://github.com/Open-ET/openet-core

#    !pip install openet-core

# Installing openet.refetgee package
# https://github.com/Open-ET/openet-refet-gee

#    !pip install openet-refet-gee

### Inicializando Google Eaeth Engine
#### Necessário definir o projeto_id a ser utilizado

In [3]:
project_id = 'et-brasil'

try:
    ee.Initialize(project=project_id)
except:
    ee.Authenticate()

### Importando o modelo SSEBop e definido a localização do ambiente de trabalho 

In [4]:
import pprint
import sys
import datetime
import pandas as pd
from IPython.display import Image

sys.path.append(r'C:\openet_brazil\workshop_et_models\openet-ssebop')

import openet.ssebop as model

### Variáveis

In [5]:
collections = ['LANDSAT/LC08/C02/T1_L2','LANDSAT/LE07/C02/T1_L2']
et_reference_source = 'projects/et-brasil/assets/era5/daily/etr'
et_reference_band = 'eto24h'
et_reference_factor = 1.0
et_reference_resample = 'nearest'
et_reference_date_type = 'daily'

# Ponto de interesse
point = ee.Geometry.Point([-47.2425,-17.5372])

# Período de análise
start_date = '2018-01-01'
end_date = '2019-01-01'

# Cobertura de nuvens
cloud_cover = 70

# Paleta de cores
ndvi_palette = ['#EFE7E1', '#003300']
et_palette = [
    'DEC29B', 'E6CDA1', 'EDD9A6', 'F5E4A9', 'FFF4AD', 'C3E683', '6BCC5C', 
    '3BB369', '20998F', '1C8691', '16678A', '114982', '0B2C7A']
viridis_palette = ['440154', '433982', '30678D', '218F8B', '36B677', '8ED542', 'FDE725']
lst_palette = ['blue','green','yellow','orange','red']

# Geometria para visualização
area_de_interesse = point.buffer(20000)

# Tamanho da imagem para a visualização
image_size = 768

### Criando o objeto do modelo

In [7]:
model_obj = model.Collection(
    collections=collections,
    et_reference_source=et_reference_source, 
    et_reference_band=et_reference_band,
    et_reference_factor=et_reference_factor,
    et_reference_resample=et_reference_resample,
    et_reference_date_type=et_reference_date_type,
    start_date=start_date,
    end_date=end_date,
    geometry=point,
    cloud_cover_max=cloud_cover,
    # filter_args={},
)
len(model_obj.get_image_ids())

32

### Opção 1 para exportar uma tabela com os resultados para um ponto (Utilizando Pandas)
#### Obs: funciona para períodos curtos e modelos simples (ssebop, ptjpl, sims)

In [9]:
def get_region_df(info):
    """Convert the output of getRegions to a pandas dataframe"""
    col_dict = {}
    info_dict = {}
    for i, k in enumerate(info[0][4:]):
        col_dict[k] = i+4
        info_dict[k] = {}
        
    for row in info[1:]:
        date = datetime.datetime.utcfromtimestamp(row[3] / 1000.0).strftime('%Y-%m-%d')
        for k, v in col_dict.items():
            info_dict[k][date] = row[col_dict[k]]
            
    return pd.DataFrame.from_dict(info_dict).sort_index()

### Criando uma classe com os resultados de et, ndbi, etr e etf

In [10]:
overpass_coll = model_obj.overpass(variables=['ndvi', 'et', 'et_reference', 'et_fraction'])

### Visualizando os resultados em uma tabela

#### Para salvar utilizar a função .to_excel() ou .to_csv()

In [11]:
overpass_df = get_region_df(overpass_coll.getRegion(point, scale=30).getInfo())
pprint.pprint(overpass_df)

#exporting to csv
overpass_df.to_csv(r'data/ssebop/ssebop_example.csv')

                ndvi        et  et_reference  et_fraction
2018-01-09  0.886899  3.755770      5.193766     0.723130
2018-01-17       NaN       NaN           NaN          NaN
2018-02-18  0.288084  0.000000      4.223090     0.000000
2018-03-06       NaN       NaN           NaN          NaN
2018-03-14  0.478672  2.900616      4.702967     0.616763
2018-03-22       NaN       NaN           NaN          NaN
2018-03-30  0.512190  2.484345      4.252199     0.584249
2018-04-23  0.577786  2.865027      2.886883     0.992429
2018-05-01  0.837812  3.038007      3.452291     0.879998
2018-05-09  0.818078  2.840740      3.197763     0.888352
2018-05-17       NaN       NaN           NaN          NaN
2018-05-25       NaN       NaN           NaN          NaN
2018-06-02  0.760818  2.997303      2.997303     1.000000
2018-06-10  0.724088  2.338674      2.934008     0.797092
2018-06-18  0.701364  2.415411      3.135368     0.770376
2018-06-26  0.585676  1.737370      2.583273     0.672546
2018-07-04  0.

C:\Users\hge2022\AppData\Local\Temp\ipykernel_22064\1689188303.py:10: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  date = datetime.datetime.utcfromtimestamp(row[3] / 1000.0).strftime('%Y-%m-%d')


### Opção 2 para exportar uma tabela com os resultados para um ponto (Utilizando Earth Engine)
#### Para períodos mais longos e modelos mais complexos (geesebal, eemetric)


In [10]:
### Função GEE para extrair dados em uma tabela

def get_table(img):
    
    reduce= img.reduceRegion(
        reducer=ee.Reducer.mean(),
        scale=90,
        maxPixels=1e12,
        geometry=point)

    date= ee.Date(img.get('system:time_start')).format('YYYY-MM-dd')
    
    return ee.Feature(None, {
    'date' : date,
    'et': ee.List([reduce.get('et'),-9999]).reduce(ee.Reducer.firstNonNull()),
    'ndvi': ee.List([reduce.get('ndvi'),-9999]).reduce(ee.Reducer.firstNonNull()),
    'et_reference ': ee.List([reduce.get('et_reference'),-9999]).reduce(ee.Reducer.firstNonNull()),
    'et_fraction ': ee.List([reduce.get('et_fraction'),-9999]).reduce(ee.Reducer.firstNonNull()),
    })


In [12]:
task = ee.batch.Export.table.toDrive(
    collection=overpass_coll.map(get_table),
    description='serie_de_dados_et',
    folder='workshop_ana',
    fileFormat='CSV',
    selectors= ['date','et','ndvi','et_reference','et_fraction']
)
task.start()

: 